In [103]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split

In [104]:
# Constants (Feel free to tweek these and observe different outcomes)
ENABLE_DISTRICT_SPLIT = False #This will ensure that schools in the same district are groupped into either trianing, validation or test. Without it they may stradle this

In [105]:
#Loading the data with pandas
math = pd.read_csv("../Data/math.csv")
read = pd.read_csv("../Data/read.csv")
business = pd.read_csv("../Data/BUSSINES.csv")
geo = pd.read_csv("../Data/geo.csv", low_memory=False)
funding = pd.read_csv("../Data/FUNDING.csv")
income = pd.read_csv("../Data/income.csv")
teachers = pd.read_csv("../Data/Teacher Ratios.csv")
title1 = pd.read_csv("../Data/title1.csv")
#Set indexs
math = math.set_index("math_ncessch")
read = read.set_index("read_ncessch")
business = business.set_index("zip_code")
funding["ncesid"] = pd.to_numeric(funding["ncesid"], errors="coerce")
funding = funding.set_index("ncesid")
geo = geo.set_index("ncessch")
income = income.set_index("zipcode")
teachers["ncessch"] = teachers["ncessch"].astype(np.int64)
teachers = teachers.set_index("ncessch")
title1["ncessch"] = title1["ncessch"].astype(np.int64)
title1 = title1.set_index("ncessch")

In [106]:
list(set(title1['ntnl_school_lunch_program_status']))

['Yes, under Community Eligibility Option (CEO)',
 'Yes, under Provision 2',
 'Yes, participating without using any Provision or the CEO',
 'No',
 'MISSING',
 'Yes, under Provision 1']

In [107]:
#Fix title1 data
title1 = title1[title1['title_i_status'] != 'M']
title1 = title1[title1['title_i_status'] != '-9']
title1 = title1[title1['title_i_eligibility'] != 'Missing']
title1 = title1[title1['title_i_eligibility'] != '-9']
title1 = title1[title1['ntnl_school_lunch_program_status'] != 'MISSING']
title1 = title1[title1['ntnl_school_lunch_program_status'] != '-9']
title1 = title1[title1['school_wide_title_i_eligibility'] != 'Missing']
title1 = title1[title1['school_wide_title_i_eligibility'] != '-9']
title1['ntnl_school_lunch_program_status_Yes'] = title1['ntnl_school_lunch_program_status'].apply(lambda x: 'Yes' in x)
title1 = pd.concat([title1.drop('title_i_eligibility',1),pd.get_dummies(title1["title_i_eligibility"], prefix='title_1_eligbility_')], axis=1)
title1 = pd.concat([title1.drop('school_wide_title_i_eligibility',1),pd.get_dummies(title1["school_wide_title_i_eligibility"], prefix='school_wide_title_i_eligibility')], axis=1)
title1 = pd.concat([title1.drop('ntnl_school_lunch_program_status',1),pd.get_dummies(title1["ntnl_school_lunch_program_status"], prefix='ntnl_school_lunch_program_status')], axis=1)

#Fix teachers
teachers= teachers[teachers['num_full_time'] >0]

In [108]:
#Combines the datasets together. This is similar to SQL's join statements but uses Pandas
combined = math[['math_leaid','math_all_grades_numvalid','math_all_grades_pctprof_low','math_all_grades_pctprof_high']].join([read[['read_all_grades_numvalid','read_all_grades_pctprof_low','read_all_grades_pctprof_high']],geo[['zip','locale','lat','lon']]], how="inner")
combined = combined.join(business[['num_establishments',"num_paid_employees", "first_quarter_payroll", "annual_payroll"]], how="inner", on="zip")
combined = combined.join(funding.drop(["idcensus","name","conum","csa","cbsa","enroll"],1), how="inner", on="math_leaid")
combined = combined.join(income.drop(["statefips",'state'], 1), how="inner", on="zip")
combined = combined.join(teachers["num_full_time"], how="inner")
combined = combined.join(title1[["title_i_status",'ntnl_school_lunch_program_status_Yes', 'title_1_eligbility__No',
       'title_1_eligbility__Not Applicable', 'title_1_eligbility__Yes',
       'school_wide_title_i_eligibility_No',
       'school_wide_title_i_eligibility_Not Applicable',
       'school_wide_title_i_eligibility_Yes',
       'ntnl_school_lunch_program_status_No',
       'ntnl_school_lunch_program_status_Yes, participating without using any Provision or the CEO',
       'ntnl_school_lunch_program_status_Yes, under Community Eligibility Option (CEO)',
       'ntnl_school_lunch_program_status_Yes, under Provision 1',
       'ntnl_school_lunch_program_status_Yes, under Provision 2']], how="inner")
combined = combined.drop(['zip','num_paid_employees',"first_quarter_payroll", "annual_payroll"],1)
combined = combined[combined['math_all_grades_pctprof_low'] >=0]
combined = combined[combined['read_all_grades_pctprof_low'] >=0]
combined['math_full_time'] = combined['math_all_grades_numvalid']/combined['num_full_time']
combined['read_full_time'] = combined['read_all_grades_numvalid']/combined['num_full_time']

In [109]:
def boxBySchoolDistrict(df):
    entryGroups = []
    groups = df.groupby('math_leaid')
    for name, entry in groups:
        entryGroups.append(entry)
    return entryGroups
def unbox(entryGroups):
    df = pd.DataFrame()
    return pd.concat(entryGroups)

# Grouping by school districts
The above code groups the data by school district to ensure that each school in a school district is either in the training, test or validation data. If this is not enabled then one school from the district may be in the training, another in the test group and one in the validation group. This is important because some of the data relies on school districts rather than individual schools this ensures that the models are learning the trends not just memorizing what happened at other schools. Untimatly, it is up to the user to determine the importance of this.

In [110]:
if ENABLE_DISTRICT_SPLIT:
    math_train_val, math_test,  = train_test_split(boxBySchoolDistrict(combined), test_size = .2, train_size = .8, random_state =42 )
else:
    math_train_val, math_test,  = train_test_split(combined, test_size = .2, train_size = .8, random_state =42 )
math_train, math_val = train_test_split(math_train_val, test_size = .3, train_size = .7, random_state =42 )
def SplitValuesMath(math):
    math = unbox(math)
    math_y = (math['math_all_grades_pctprof_low']+math['math_all_grades_pctprof_high'])/2
    math_x = math.drop(["math_all_grades_pctprof_low","math_all_grades_pctprof_high","read_all_grades_pctprof_low","read_all_grades_pctprof_high"],1)
    return math_x, math_y
math_X_train, math_y_train = SplitValuesMath(math_train)
math_X_val, math_y_val = SplitValuesMath(math_val)
math_X_test, math_y_test = SplitValuesMath(math_test)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
#depth = 0
#max_score = 0
#for i in range(1,20):
#    print("running with max depth ", i)
#    rfr = RandomForestRegressor(n_estimators=100, max_depth=i, random_state=42)
#    trained = rfr.fit(math_X_train, math_y_train)
#    score = rfr.score(math_X_val, math_y_val)
#    print("validation score", score)
#    if score > max_score:
#        depth = i
#        max_score = score

#Optimal at 19 max depth
rfr = RandomForestRegressor(n_estimators=100, max_depth=19, random_state=42)
trained = rfr.fit(math_X_train, math_y_train)
print(rfr.score(math_X_test, math_y_test))

In [ ]:
def calculateDelta(x_test, y_test,estimator):
    predictions = np.array(estimator.predict(x_test))
    return np.mean(np.absolute(predictions - y_test))
calculateDelta(math_X_test, math_y_test, rfr)

In [ ]:
features = pd.DataFrame({'scores':rfr.feature_importances_, 'names':math_X_train.columns})
features.sort_values(by='scores', ascending=False)